In [1]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import argparse
from copy import deepcopy
import torch
import gym
import time
import torch
import torch.nn as nn	# 各种层类型的实现
import torch.nn.functional as F	# 各中层函数的实现，与层类型对应，如：卷积函数、池化函数、归一化函数等等
import torch.optim as optim	# 实现各种优化算法的包
import numpy as np

import argparse
import sys


import numpy as np
from numpy.random import permutation
import matplotlib.pyplot as plt
from grid_data import GridData

from compute_mpe import CircuitMPE
import sys
sys.path.insert(0,'/root/gurobi903/linux64/lib/python3.6_utf32/gurobipy')
import gurobipy
cmpe = CircuitMPE('/root/PySDD/notebooks/yahoo.vtree', '/root/PySDD/notebooks/yahoo.sdd')
class para:
    def __init__(self):
        self.batch_size=16
        self.wmc=10
        self.solDim=24
        self.card=10
        self.test_batch_size=1000
        self.epochs=10
        self.lr=0.1
        self.momentum=0.5
        self.no_cuda='store_true'
        self.seed=1
        self.log_interval=10
        self.hidden_dim=10
args=para()
legalList={0: [13, 18],
 1: [17, 20, 22],
 2: [4, 5, 10, 13, 14, 16, 17, 18, 20, 22],
 3: [5, 9, 11, 12, 15],
 4: [2, 10, 13, 14, 16, 17, 18, 20, 22],
 5: [2, 3, 6, 9, 10, 11, 12, 14, 16],
 6: [5, 15],
 7: [],
 8: [],
 9: [3, 5, 11, 12, 15],
 10: [2, 4, 5, 14, 16, 22],
 11: [3, 5, 9, 12, 15],
 12: [3, 5, 9, 11, 15],
 13: [0, 2, 4, 14, 18, 22],
 14: [2, 4, 5, 10, 13, 16, 17, 18, 20, 22],
 15: [3, 6, 9, 11, 12, 17, 19, 20],
 16: [2, 4, 5, 10, 14, 22],
 17: [1, 2, 4, 14, 15, 19, 20, 22],
 18: [0, 2, 4, 13, 14, 22],
 19: [15, 17, 20],
 20: [1, 2, 4, 14, 15, 17, 19, 22],
 21: [],
 22: [1, 2, 4, 10, 13, 14, 16, 17, 18, 20],
 23: []}
def checkFea(x):
    cnt=0
    cntT=0
    for i in range(len(legalList.keys())):
        for j in legalList[i]:
            cntT+=1
            if x[j]+x[i]>1:
                cnt+=1
    return cnt/cntT/2#+abs(x.sum()-args.card)/args.card/2
def CB(alpha,x,M):
    return alpha*np.sqrt(np.dot(np.dot(x.T,np.linalg.inv(M)),x))
def solver(theta,MODEL):

    import gurobipy
    import time
    s=time.time()
    # 创建模型
    variables=[]
    for i in range(args.solDim):
        variables.append(MODEL.addVar(vtype=gurobipy.GRB.BINARY, name='x'+'i'))
    # 更新变量环境
    MODEL.update()

    # 创建目标函数
    MODEL.setObjective(np.array(variables).dot(theta), sense=gurobipy.GRB.MAXIMIZE)

    # 创建约束条件
    cnt=0
    for i in range(args.solDim):
        if len(legalList)>0:
            for j in legalList[i]:
                MODEL.addConstr(-variables[i]-variables[j] >= -1, name=str(cnt))
                cnt+=1
    MODEL.addConstr(sum(variables) == args.card, name=str(cnt))
    f=time.time()
    # 执行最优化
    MODEL.optimize()
    return np.array(MODEL.x)
def solver_quad(Q,MODEL):

    import gurobipy
    import time
    s=time.time()
    # 创建模型
    variables=[]
    for i in range(args.solDim):
        variables.append(MODEL.addVar(vtype=gurobipy.GRB.BINARY, name='x'+'i'))
    # 更新变量环境
    MODEL.update()

    # 创建目标函数
    MODEL.setObjective(np.array(variables).dot(Q).dot(np.array(variables)), sense=gurobipy.GRB.MAXIMIZE)

    # 创建约束条件
    cnt=0
    for i in range(args.solDim):
        if len(legalList)>0:
            for j in legalList[i]:
                MODEL.addConstr(-variables[i]-variables[j] >= -1, name=str(cnt))
                cnt+=1
    MODEL.addConstr(sum(variables) == args.card, name=str(cnt))
    f=time.time()
    # 执行最优化
    
    print('??')
    MODEL.optimize()
    print('??')
    return np.array(MODEL.x)
state_value=100

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional
import torch.nn.functional as F

def init(module, weight_init, bias_init, gain=1):
    weight_init(module.weight.data, gain=gain)
    bias_init(module.bias.data)
    return module

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

def layer_init(layer, w_scale=1.0):
    nn.init.orthogonal_(layer.weight.data)
    layer.weight.data.mul_(w_scale)
    nn.init.constant_(layer.bias.data, 0)
    return layer
def fanin_init(size, fanin=None):
        fanin = fanin or size[0]
        v = 1. / np.sqrt(fanin)
        return torch.Tensor(size).uniform_(-v, v)

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(24, 400)
        self.fc2 = nn.Linear(400, 300)
        self.fc3 = nn.Linear(300, 24)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.init_weights()
        self.sigmoid=nn.Sigmoid()
    def init_weights(self):
        self.fc1.weight.data = fanin_init(self.fc1.weight.data.size())
        self.fc2.weight.data = fanin_init(self.fc2.weight.data.size())
        self.fc3.weight.data.uniform_(-3e-1, 3e-1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.tanh(out)
        out=self.sigmoid(out)
        return out

In [11]:
import numpy as np
import time
import random
from gym import spaces
import gym
from collections import defaultdict
from sklearn.preprocessing import normalize

import torch
import torch.nn as nn
import torch.optim
import torch.nn.functional
import torch.nn.functional as F
from torch.distributions import Categorical,Normal

from collections import namedtuple
import itertools
import copy
SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])
class RL_Oracle_Actor_Critic:
    def __init__(self, env=None, theta=None, net=None, args=None):
        super(RL_Oracle_Actor_Critic, self).__init__()
        self.env = env
        self.gamma = args.gamma
        self.lr = args.rl_lr
        self.saved_actions = []
        self.rewards = []
        self.entropies = []
        self.theta = theta
        self.eps = np.finfo(np.float32).eps.item()
        self.device = args.device

        self.net = net
        self.optimizer = torch.optim.Adam(self.net.parameters(), lr=self.lr)#
        #self.optimizer = torch.optim.RMSprop(self.net.parameters(), lr=self.lr, eps=1e-5, alpha=0.99)
        self.entropy_coef = args.entropy_coef
        self.value_coef = args.value_coef
        #self.diversity = args.diversity

    def reset(self, normalize_theta=True):
        del self.saved_actions[:]
        del self.rewards[:]
        del self.entropies[:]

    def select_action(self, state):
        state = state.to(self.device)
        action = self.net(state)
        self.optimizer.zero_grad()
        loss=sum(action)
        loss.backward()
        m=torch.distributions.bernoulli.Bernoulli(torch.tensor(action))
        self.saved_actions.append(SavedAction(sum(m.entropy()), state_value))
        return action

    def finish_episode(self, normalize_theta=None):
        R = 0
        saved_actions = self.saved_actions
        policy_losses = []
        value_losses = []
        returns = []
        for r in self.rewards[::-1]:
            R = r + self.gamma * R
            returns.insert(0, R)
        returns = torch.tensor(returns)

        #returns = torch.tensor(returns)
        #returns = (returns - returns.mean()) / (returns.std() + self.eps)
        for (log_prob, value), R in zip(saved_actions, returns):
            advantage = -R #- value
            policy_losses.append(-1* advantage)
            R = torch.tensor([R]).to(self.device)
            value_losses.append(F.smooth_l1_loss(torch.tensor(value), R.reshape(-1,1)))
        self.optimizer.zero_grad()
        loss=policy_losses[-1]#+self.value_coef *value_losses[-1]
        print(loss)
        #loss =( torch.stack(policy_losses).mean().float()\
         #        + (self.value_coef * torch.stack(value_losses).mean().float()))
                # - (self.entropy_coef * torch.stack(self.entropies).mean()).float())
        loss=torch.tensor(loss, dtype=torch.float32,requires_grad=True)
        loss.backward()
        nn.utils.clip_grad_norm_(self.net.parameters(), 0.5)
        self.optimizer.step()
        self.reset(normalize_theta=normalize_theta)

    def learn_policy(self, n_traj=50, n_iter=300, update=True, normalize_theta=True, cost=False):
        sum_measurements = np.zeros(np.shape(self.theta))
        episode_stats = defaultdict(list)

        for _ in range(n_traj):
            obs = torch.from_numpy(np.array([0.02]*24).astype(np.float32))
            done = False
            traj_measurements = np.zeros(self.theta.size)

            for i in range(n_iter):
                action = self.select_action(obs)
                
                obs=action.detach().numpy()
                done=1
                yu=torch.unbind(action)
                xu = torch.unbind(torch.from_numpy(np.array([0.02]*24).astype(np.float32)))
                wmc = cmpe.get_torch_ac([[1.0 - ny,ny] for ny in yu + xu]) 
                a = 10*(np.load('rateListUsersYahoo.npy')[0]-np.load('rateListUsersYahoo.npy')[0].min())#np.random.random(24) #  np.load('rateListUsersYahoo.npy')[0]
                a /= np.linalg.norm(a, ord=2)
                env_reward  =torch.dot(action,torch.from_numpy(a).float()).item()
#                print(env_reward)          
                info={'goal': 1, 'constraint': -torch.log(torch.mean(wmc)).float()}
                constraint = info['constraint']
                goal = info['goal']
                measurements = np.array([-float(constraint), env_reward])
                traj_measurements = traj_measurements + measurements
                reward = np.dot(self.theta, measurements)
                if cost:
                    reward = -reward
                self.rewards.append(reward)
                self.optimizer.zero_grad()
                loss=reward
                #loss =( torch.stack(policy_losses).mean().float()\
                 #        + (self.value_coef * torch.stack(value_losses).mean().float()))
                        # - (self.entropy_coef * torch.stack(self.entropies).mean()).float())
                loss=torch.tensor(loss, dtype=torch.float32,requires_grad=True)
                loss.backward()
                if done:
                    break

            episode_stats['traj_len'].append(i)
            if update:
                self.finish_episode(normalize_theta=normalize_theta)

            sum_measurements = sum_measurements+traj_measurements

            episode_stats['goals'].append(goal * 1)

        #print(f'{obs}')
        #print('-----')
        avg_measurements = sum_measurements / n_traj
        return (avg_measurements, episode_stats)

In [16]:
  
import torch
import random
import numpy as np
import argparse

RANDOM_SEED = random.randint(0,1e+5)

torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
class appropo_args:
    def __init__(self):
        self.num_epochs=3000
        self.cache_size=5
        self.output='results'
        self.print='store_true'
        self.diversity='store_true'
        self.threshold=0.12
        self.check_traj=5
        self.device='cpu'
        self.mx_size=20
        self.name='tmp'
        self.init_variable='constraint'# choices=['constraint', 'reward', 'both','None'])

    # RL Oracle
        self.rl_iter=300
        self.rl_traj=20
        self.rl_lr=1e-2
        self.entropy_coef=.001
        self.value_coef=1.0
        self.gamma=0.99
    # Projection Oracle
        self.proj_lr=1
        self.olo_optim='sgd'
        self.atol_cache=0.0
        self.atol_proj=1e-1
        self.rtol_cache=1e-05
        self.rtol_proj=1e-05
        self.prob_failure=0.01

    # RCPO
        self.num_episodes=1000000
        self.lambda_lr=0.000025
        self.constraint=.20
        self.log_interval=1#, metavar='N',   help='interval between training status logs (default: 10)')

    # Frozen Mars Rover Environment
        self.map='8x8'#, choices=['4x4', '8x8'], default='8x8')
args=appropo_args()

In [17]:
from datetime import datetime
import argparse
import random
import pandas as pd
import gym
import numpy as np
import torch
from collections import defaultdict

In [18]:
from collections import deque

class MixturePolicy:
    def __init__(self):
        self.goals = []

        self.loss_vec = []
        self.stats = []

    def add_response(self, best_exp_rtn=None, traj_len=None, stats=None):
        self.loss_vec.append(best_exp_rtn)
        self.exp_rtn_of_avg_policy = np.average(np.stack(self.loss_vec, axis=0), axis=0)

        try:
            self.goals.append(stats['goals'])
            self.exp_rtn_goal = np.average(self.goals)
        except:
            raise Exception('Error: goal is not being passsed')

        self.stats.append(np.hstack([stats['episode'], stats['num_trajs'],\
              self.exp_rtn_of_avg_policy[1], self.exp_rtn_of_avg_policy[0],\
              0,\
              stats['oracle_calls'], stats['cache_calls'], {'lens': traj_len}, np.mean(traj_len),\
              stats['num_samples']]))

class RCPOPolicy:
    def __init__(self, shadow=False):
        self.goals = []
        self.cache_calls = 0
        self.oracle_calls = 0
        self.num_samples = 0
        self.num_trajs = 0
        self.stats = []

    def add_response(self, episode_stats=None, episode=None, rtn=None):
        self.num_samples += sum(episode_stats['traj_len'])
        self.num_trajs += len(episode_stats['traj_len'])
        self.oracle_calls += 1

        self.stats.append(np.hstack([episode, self.num_trajs, rtn[1], rtn[0], 0,\
            self.oracle_calls, self.cache_calls, {'lens': episode_stats['traj_len']},\
            np.mean(episode_stats['traj_len']), self.num_samples, episode_stats['_lambda'][0] ]))


In [19]:

# Fix Random seeds in all the underlying libraries
RANDOM_SEED = random.randint(0,1e+5)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

theta = np.zeros(2 )
theta[1] = 1
if args.init_variable== 'constraint':
    theta[0] = _lambda = 2
else:
    theta[0] = _lambda = 0.0

stats = defaultdict(int)

net = MLP()
rl_oracle = RL_Oracle_Actor_Critic(net=net, args=args, theta=theta)
total_num_samples = 25000

for episode in range(args.num_episodes):
    [rtn, episode_stats] = rl_oracle.learn_policy(n_traj=1, n_iter=args.rl_iter,\
                                             normalize_theta=False, update=True)
    _lambda = max(0, _lambda + args.lambda_lr*(-args.prob_failure-rtn[0]))

    theta[0] = _lambda
    rl_oracle.theta =  theta

    
    if stats['num_samples'] > total_num_samples:
        break



/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:72: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:79: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0.5557, 0.4834, 0.4945, 0.4716, 0.5416, 0.4671, 0.4480,
        0.5047, 0.5335, 0.5294, 0.5288, 0.4824, 0.5140],
       grad_fn=<SigmoidBackward>)
tensor(-12.7427, dtype=torch.float64)
action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0.5557, 0.4834, 0.4945, 0.4716, 0.5416, 0.4671, 0.4480,
        0.5047, 0.5335, 0.5294, 0.5288, 0.4824, 0.5140],
       grad_fn=<SigmoidBackward>)
tensor(-12.7441, dtype=torch.float64)
action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0.5557, 0.4834, 0.4945, 0.4716, 0.5416, 0.4671, 0.4480,
        0.5047, 0.5335, 0.5294, 0.5288, 0.4824, 0.5140],
       grad_fn=<SigmoidBackward>)
tensor(-12.7455, dtype=torch.float64)
action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0

action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0.5557, 0.4834, 0.4945, 0.4716, 0.5416, 0.4671, 0.4480,
        0.5047, 0.5335, 0.5294, 0.5288, 0.4824, 0.5140],
       grad_fn=<SigmoidBackward>)
tensor(-12.7977, dtype=torch.float64)
action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0.5557, 0.4834, 0.4945, 0.4716, 0.5416, 0.4671, 0.4480,
        0.5047, 0.5335, 0.5294, 0.5288, 0.4824, 0.5140],
       grad_fn=<SigmoidBackward>)
tensor(-12.7991, dtype=torch.float64)
action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0.5557, 0.4834, 0.4945, 0.4716, 0.5416, 0.4671, 0.4480,
        0.5047, 0.5335, 0.5294, 0.5288, 0.4824, 0.5140],
       grad_fn=<SigmoidBackward>)
tensor(-12.8005, dtype=torch.float64)
action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0

action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0.5557, 0.4834, 0.4945, 0.4716, 0.5416, 0.4671, 0.4480,
        0.5047, 0.5335, 0.5294, 0.5288, 0.4824, 0.5140],
       grad_fn=<SigmoidBackward>)
tensor(-12.8570, dtype=torch.float64)
action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0.5557, 0.4834, 0.4945, 0.4716, 0.5416, 0.4671, 0.4480,
        0.5047, 0.5335, 0.5294, 0.5288, 0.4824, 0.5140],
       grad_fn=<SigmoidBackward>)
tensor(-12.8584, dtype=torch.float64)
action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0.5557, 0.4834, 0.4945, 0.4716, 0.5416, 0.4671, 0.4480,
        0.5047, 0.5335, 0.5294, 0.5288, 0.4824, 0.5140],
       grad_fn=<SigmoidBackward>)
tensor(-12.8598, dtype=torch.float64)
action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0

action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0.5557, 0.4834, 0.4945, 0.4716, 0.5416, 0.4671, 0.4480,
        0.5047, 0.5335, 0.5294, 0.5288, 0.4824, 0.5140],
       grad_fn=<SigmoidBackward>)
tensor(-12.9162, dtype=torch.float64)
action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0.5557, 0.4834, 0.4945, 0.4716, 0.5416, 0.4671, 0.4480,
        0.5047, 0.5335, 0.5294, 0.5288, 0.4824, 0.5140],
       grad_fn=<SigmoidBackward>)
tensor(-12.9177, dtype=torch.float64)
action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0.5557, 0.4834, 0.4945, 0.4716, 0.5416, 0.4671, 0.4480,
        0.5047, 0.5335, 0.5294, 0.5288, 0.4824, 0.5140],
       grad_fn=<SigmoidBackward>)
tensor(-12.9191, dtype=torch.float64)
action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0

...

action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0.5557, 0.4834, 0.4945, 0.4716, 0.5416, 0.4671, 0.4480,
        0.5047, 0.5335, 0.5294, 0.5288, 0.4824, 0.5140],
       grad_fn=<SigmoidBackward>)
tensor(-13.0319, dtype=torch.float64)
action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0.5557, 0.4834, 0.4945, 0.4716, 0.5416, 0.4671, 0.4480,
        0.5047, 0.5335, 0.5294, 0.5288, 0.4824, 0.5140],
       grad_fn=<SigmoidBackward>)
tensor(-13.0334, dtype=torch.float64)
action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0.5557, 0.4834, 0.4945, 0.4716, 0.5416, 0.4671, 0.4480,
        0.5047, 0.5335, 0.5294, 0.5288, 0.4824, 0.5140],
       grad_fn=<SigmoidBackward>)
tensor(-13.0348, dtype=torch.float64)
action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0

tensor(-13.0884, dtype=torch.float64)
action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0.5557, 0.4834, 0.4945, 0.4716, 0.5416, 0.4671, 0.4480,
        0.5047, 0.5335, 0.5294, 0.5288, 0.4824, 0.5140],
       grad_fn=<SigmoidBackward>)
tensor(-13.0898, dtype=torch.float64)
action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0.5557, 0.4834, 0.4945, 0.4716, 0.5416, 0.4671, 0.4480,
        0.5047, 0.5335, 0.5294, 0.5288, 0.4824, 0.5140],
       grad_fn=<SigmoidBackward>)
tensor(-13.0912, dtype=torch.float64)
action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0.5557, 0.4834, 0.4945, 0.4716, 0.5416, 0.4671, 0.4480,
        0.5047, 0.5335, 0.5294, 0.5288, 0.4824, 0.5140],
       grad_fn=<SigmoidBackward>)
tensor(-13.0926, dtype=torch.float64)
action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4

action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0.5557, 0.4834, 0.4945, 0.4716, 0.5416, 0.4671, 0.4480,
        0.5047, 0.5335, 0.5294, 0.5288, 0.4824, 0.5140],
       grad_fn=<SigmoidBackward>)
tensor(-13.1476, dtype=torch.float64)
action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0.5557, 0.4834, 0.4945, 0.4716, 0.5416, 0.4671, 0.4480,
        0.5047, 0.5335, 0.5294, 0.5288, 0.4824, 0.5140],
       grad_fn=<SigmoidBackward>)
tensor(-13.1491, dtype=torch.float64)
action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0.5557, 0.4834, 0.4945, 0.4716, 0.5416, 0.4671, 0.4480,
        0.5047, 0.5335, 0.5294, 0.5288, 0.4824, 0.5140],
       grad_fn=<SigmoidBackward>)
tensor(-13.1505, dtype=torch.float64)
action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0

action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0.5557, 0.4834, 0.4945, 0.4716, 0.5416, 0.4671, 0.4480,
        0.5047, 0.5335, 0.5294, 0.5288, 0.4824, 0.5140],
       grad_fn=<SigmoidBackward>)
tensor(-13.2069, dtype=torch.float64)
action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0.5557, 0.4834, 0.4945, 0.4716, 0.5416, 0.4671, 0.4480,
        0.5047, 0.5335, 0.5294, 0.5288, 0.4824, 0.5140],
       grad_fn=<SigmoidBackward>)
tensor(-13.2083, dtype=torch.float64)
action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0.5557, 0.4834, 0.4945, 0.4716, 0.5416, 0.4671, 0.4480,
        0.5047, 0.5335, 0.5294, 0.5288, 0.4824, 0.5140],
       grad_fn=<SigmoidBackward>)
tensor(-13.2097, dtype=torch.float64)
action tensor([0.4387, 0.4877, 0.4774, 0.5024, 0.4843, 0.4744, 0.5147, 0.4922, 0.4628,
        0.5281, 0.5466, 0

KeyboardInterrupt: 

In [ ]:
theta